In [1]:
import torch 
import json,pickle,math
import pandas as pd
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
LSM = pickle.load(open('./davis_ligand_similarity_matrix.pkl', 'rb'))
PSM = pickle.load(open('./davis_protein_similarity_matrix.pkl', 'rb'))
full_df = pd.read_csv(open('./davis_all_pairs.csv','r'))

In [3]:
# df

In [4]:
SMILES = json.load(open('./data/DAVIS/SMILES.txt'))
TARGETS = json.load(open('./data/DAVIS/target_seq.txt'))
SMILES=list(SMILES.values())
TARGETS=list(TARGETS.values())

In [5]:
all_9_folds={}
for i in [0,1,2]:
    for j in [0,1,2]:
        file_name = 'fold' +str(i) +str(j) 
        
        temp = open('./data/davis/DAVIS_9_FOLDS/' + file_name +'.pkl', 'rb')
        new_df = pd.read_pickle(temp)
        all_9_folds.update({file_name:new_df})
        temp.close()
        

In [6]:
def create_davis_test_train(test_fold_number,all_9_folds):
    test_protein_fold_id = test_fold_number[0]
    test_ligand_fold_id = test_fold_number[1]
    test_set = pd.DataFrame(columns = full_df.columns)
    train_set = pd.DataFrame(columns= full_df.columns)
    for i in [0,1,2]:
        for j in [0,1,2]:
            fold_name = 'fold' + str(i) + str(j)
            df = all_9_folds[fold_name]
            
            if str(i) == test_protein_fold_id and str(j) == test_ligand_fold_id:
                test_set = df.copy()
                
            if str(i) != test_protein_fold_id and str(j) != test_ligand_fold_id:
                train_set = pd.concat([train_set, df.copy()], ignore_index=True)
                
                
    return train_set, test_set


# Create train test split on these 9 folds
## fold_number is the id of fold. For example, test = fold00, train = fold 11,22,12,21

In [7]:
fold_number = '01'

In [8]:
train, test = create_davis_test_train(test_fold_number=fold_number, all_9_folds=all_9_folds)

# To ensure that there are no common targets or drugs in train and test


In [9]:
test_smiles = list(test['SMILES'])
test_targets = list(test['Target Sequence'])
train_smiles = list(train['SMILES'])
train_targets = list(train['Target Sequence'])

for i in test_smiles:
    if i in train_smiles:
        print("common entity present")
for i in test_targets:
    if i in train_targets:
        print("common entity present")


# Creating outer products for test set

In [10]:
outer_test_prods = []
for i,row in test.iterrows():
#     print(i)
    smi = row['SMILES']
    seq = row['Target Sequence']
    target_id = TARGETS.index(seq)
    smi_id = SMILES.index(smi)
    ki=LSM[smi_id]
    kj=PSM[target_id]
    ki_x_kj = np.outer(ki,kj)
    outer_test_prods.append([ki_x_kj])
outer_test_prods = np.array(outer_test_prods)
print(np.shape(outer_test_prods))

(3404, 1, 68, 442)


# creating outer products for train set

In [11]:
outer_train_prods = []
for i,row in train.iterrows():
#     print(i)
    smi = row['SMILES']
    seq = row['Target Sequence']
    target_id = TARGETS.index(seq)
    smi_id = SMILES.index(smi)
    ki=LSM[smi_id]
    kj=PSM[target_id]
    ki_x_kj = np.outer(ki,kj)
    outer_train_prods.append([ki_x_kj])
outer_train_prods = np.array(outer_train_prods)
print(np.shape(outer_train_prods))

(13230, 1, 68, 442)


In [12]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 20
# num_classes = 10
batch_size = 32
learning_rate = 0.001

In [13]:
class custom_dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, outer_prods, transform=None):
#         self.df = pd.read_csv(open(csv_file))
        self.df = dataframe
#         self.root_dir = root_dir
        self.transform = transform
        self.outer_prods = outer_prods
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        output = {'outer_product': self.outer_prods[idx] , 'Label':self.df.iloc[idx]['Label']}
        return output

In [14]:
train_dataset = custom_dataset(dataframe = train, outer_prods = outer_train_prods)
test_dataset = custom_dataset(dataframe = test, outer_prods = outer_test_prods)


In [15]:
train_loader= torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader= torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [16]:
print(len(train_loader)*32, len(test_loader)*32)

13248 3424


In [17]:
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1,32, 5).double()
        self.pool1 = nn.MaxPool2d(2,2).double()
        self.conv2 = nn.Conv2d(32,18,3).double()
        self.pool2 = nn.MaxPool2d(2,2).double()
        self.fc1 = nn.Linear(18*15*108, 128).double()
        self.fc2 = nn.Linear(128,1).double()
        self.dropout = nn.Dropout(0.1).double()
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,18*15*108)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        
        return x
    

In [18]:
# for i in test_loader:
#     a = i['outer_product']
#     b= i['Label']
#     break
# conv1 = nn.Conv2d(1,32,5).double()
# pool = nn.MaxPool2d(2,2).double()
# conv2 = nn.Conv2d(32,18,3).double()
# fc1 = nn.Linear(18*15*108, 128).double()
# fc2 = nn.Linear(128,1).double()
# dropout = nn.Dropout(0.1).double()
# x= conv1(a)
# print(x.shape)
# x = pool(x)
# print(x.shape)
# x= conv2(x)
# print(x.shape)
# x = pool(x)
# print(x.shape)
# x = x.view(-1,18*15*108)
# print(x.shape)
# x = dropout(x)
# print(x.shape)
# x = fc1(x)
# print(x.shape)
# x = fc2(x)
# print(x.shape)

In [19]:
model = ConvNet().to(device)

In [20]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Evaluation metrics

In [21]:
def rmse(y,f):
    rmse = math.sqrt(((y - f)**2).mean(axis=0))
    return rmse
def mse(y,f):
    mse = ((y - f)**2).mean(axis=0)
    return mse
def pearson(y,f):
    rp = np.corrcoef(y, f)[0,1]
    return rp
from lifelines.utils import concordance_index
def ci(y,f):
    return concordance_index(y,f)

In [22]:
def predicting(model, device, test_loader):
    model.eval()
    total_preds = np.array([])
    total_labels = np.array([])
    with torch.no_grad():
        for i in test_loader:
            images = i['outer_product']
            labels = i['Label']
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images) 
            outputs = outputs.cpu().detach().numpy().flatten()
            labels =labels.cpu().detach().numpy().flatten()
            total_preds = np.concatenate([total_preds, outputs])
            total_labels = np.concatenate([total_labels, labels])
    
    model.train()
    return total_labels, total_preds

# Train the model

In [23]:
model_file_name = './SAVED_MODELS/best_sim-CNN-DTA_davis_fold' + fold_number +  '.model'
result_file_name = './SAVED_MODELS/best_result_sim-CNNDTA_davis_fold'+fold_number + '.csv'

In [ ]:
# Train the model
best_mse = 1000
best_ci = 0

total_step = len(train_loader)
for epoch in range(num_epochs):
    c=0
    for i in train_loader:
        c=c+1
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.flatten(), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, c, total_step, loss.item()))
    
    # taking best model so far
    G,P = predicting(model, device, test_loader)
    ret = [rmse(G, P), mse(G, P), pearson(G, P), ci(G, P)]
    if ret[1] < best_mse:
        torch.save(model.state_dict(), model_file_name)
        with open(result_file_name, 'w') as f:
            f.write(','.join(map(str, ret)))
        best_epoch = epoch+1
        best_mse = ret[1]
        best_ci = ret[-1]
        best_r = ret[2]
        
        print('rmse improved at epoch ', best_epoch,
                      '; best_mse,best_ci,best_r:', best_mse, best_ci,best_r)
        
        

/home2/kanakala.ganesh/miniconda3/envs/geometric/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [1/20], Step [1/414], Loss: 30.1050
Epoch [1/20], Step [2/414], Loss: 14.3208
Epoch [1/20], Step [3/414], Loss: 1.9628
Epoch [1/20], Step [4/414], Loss: 15.5687
Epoch [1/20], Step [5/414], Loss: 5.3748
Epoch [1/20], Step [6/414], Loss: 1.1842
Epoch [1/20], Step [7/414], Loss: 1.8497
Epoch [1/20], Step [8/414], Loss: 3.3616
Epoch [1/20], Step [9/414], Loss: 3.8290
Epoch [1/20], Step [10/414], Loss: 3.8511
Epoch [1/20], Step [11/414], Loss: 1.9314
Epoch [1/20], Step [12/414], Loss: 0.7047
Epoch [1/20], Step [13/414], Loss: 0.4676
Epoch [1/20], Step [14/414], Loss: 1.2022
Epoch [1/20], Step [15/414], Loss: 1.6717
Epoch [1/20], Step [16/414], Loss: 2.1351
Epoch [1/20], Step [17/414], Loss: 1.5269
Epoch [1/20], Step [18/414], Loss: 1.8639
Epoch [1/20], Step [19/414], Loss: 1.2276
Epoch [1/20], Step [20/414], Loss: 1.5450
Epoch [1/20], Step [21/414], Loss: 1.1548
Epoch [1/20], Step [22/414], Loss: 1.6819
Epoch [1/20], Step [23/414], Loss: 0.7274
Epoch [1/20], Step [24/414], Loss: 0.771

Epoch [1/20], Step [196/414], Loss: 0.3065
Epoch [1/20], Step [197/414], Loss: 0.4238
Epoch [1/20], Step [198/414], Loss: 0.4194
Epoch [1/20], Step [199/414], Loss: 0.7669
Epoch [1/20], Step [200/414], Loss: 1.8194
Epoch [1/20], Step [201/414], Loss: 0.6881
Epoch [1/20], Step [202/414], Loss: 0.7249
Epoch [1/20], Step [203/414], Loss: 0.6969
Epoch [1/20], Step [204/414], Loss: 0.3435
Epoch [1/20], Step [205/414], Loss: 1.5128
Epoch [1/20], Step [206/414], Loss: 0.9814
Epoch [1/20], Step [207/414], Loss: 0.8912
Epoch [1/20], Step [208/414], Loss: 0.5105
Epoch [1/20], Step [209/414], Loss: 0.7950
Epoch [1/20], Step [210/414], Loss: 0.7636
Epoch [1/20], Step [211/414], Loss: 0.9274
Epoch [1/20], Step [212/414], Loss: 0.4985
Epoch [1/20], Step [213/414], Loss: 0.4396
Epoch [1/20], Step [214/414], Loss: 1.0070
Epoch [1/20], Step [215/414], Loss: 0.4322
Epoch [1/20], Step [216/414], Loss: 0.6054
Epoch [1/20], Step [217/414], Loss: 0.8688
Epoch [1/20], Step [218/414], Loss: 0.7744
Epoch [1/20

In [ ]:
best_epoch

In [ ]:
model.eval()
# eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
total_preds = np.array([])
total_labels = np.array([])
with torch.no_grad():
    correct = 0
    total = 0
    for i in test_loader:
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images) 
        outputs = outputs.cpu().detach().numpy().flatten()
        labels =labels.cpu().detach().numpy().flatten()
        total_preds = np.concatenate([total_preds, outputs])
        total_labels = np.concatenate([total_labels, labels])
#         total_preds = torch.cat(total_preds, outputs.cpu(), 0 )
#         total_labels = torch.cat(total_labels, labels.cpu(), 0)
#         break

In [ ]:
G,P = total_labels, total_preds

In [ ]:
rmse(G,P)

In [ ]:
mse(G,P)

In [ ]:
pearson(G,P)

In [ ]:
ci(G,P)